### Informacoes sobre Covid-19 no Brasil dos ultimos 6 meses via API 

#### Bibliotecas Utilizadas

In [1]:
import requests
import pandas as pd
from pandas import DataFrame
from datetime import date, timedelta

#### Extracao de dados

In [2]:
#documentacao api [https://covid19-brazil-api-docs.vercel.app] 
#serviço -> [Lista casos no brasil em data específica]

#envoca API dia por dia (ultimos 6 meses)
today = date.today()
start_date = today + timedelta(days= -180)
url = "https://covid19-brazil-api.now.sh/api/report/v1/brazil/"

#criacao de dicionario para cada uf
result_dict = {}

while start_date < today:
  day = start_date.strftime("%Y%m%d")
  #print("Starting invocation from day",day)
  #retorna os dados(json) da api do dia especifico
  response = requests.get(url + day).json()

  #cada item do response.data retornara os registros pela uf
  for item in response['data']:
    uf = item['uf']
    #Nao havendo um dicionario para a uf(chave) -> add um dicionario a ela com array vazio (1º registros)
    if (not uf in result_dict):
      result_dict[uf] = []
    #e entao adiciona todos os registros correspondentes da uf em seu dicionario
    result_dict[uf].append(item)
  
  #add os resultados da api dia por dia
  start_date = start_date + timedelta(days= 1)

In [3]:
result_dict

{'SP': [{'uid': 35,
   'uf': 'SP',
   'state': 'São Paulo',
   'cases': 1418491,
   'deaths': 45758,
   'suspects': 5334,
   'refuses': 596,
   'datetime': '2020-12-24T19:06:09.179Z'},
  {'uid': 35,
   'uf': 'SP',
   'state': 'São Paulo',
   'cases': 1422087,
   'deaths': 45795,
   'suspects': 5334,
   'refuses': 596,
   'datetime': '2020-12-25T21:45:45.224Z'},
  {'uid': 35,
   'uf': 'SP',
   'state': 'São Paulo',
   'cases': 1423340,
   'deaths': 45808,
   'suspects': 5334,
   'refuses': 596,
   'datetime': '2020-12-26T21:45:55.252Z'},
  {'uid': 35,
   'uf': 'SP',
   'state': 'São Paulo',
   'cases': 1426176,
   'deaths': 45863,
   'suspects': 5334,
   'refuses': 596,
   'datetime': '2020-12-27T21:46:24.258Z'},
  {'uid': 35,
   'uf': 'SP',
   'state': 'São Paulo',
   'cases': 1427752,
   'deaths': 45902,
   'suspects': 5334,
   'refuses': 596,
   'datetime': '2020-12-28T21:47:37.199Z'},
  {'uid': 35,
   'uf': 'SP',
   'state': 'São Paulo',
   'cases': 1440229,
   'deaths': 46195,
   '

#### Tratamento e Limpeza de Dados

In [5]:
covid_df = None
#criacao de data frame para cada uf do dicionario
for uf_key in result_dict:
  new_df = pd.DataFrame(result_dict[uf_key])

  #criacao de novas colunas de casos e mortes com a contagem do dia anterior
  #zerando as colunas com NAN
  new_df['casesday'] = new_df['cases'].diff().fillna(0)
  new_df['deathsday'] = new_df['deaths'].diff().fillna(0)

  #carregamento de dados ao dataframe uf por uf 
  if covid_df is None:
    covid_df = new_df
  else:
    covid_df = pd.concat([covid_df, new_df])

In [6]:
covid_df

,uid,uf,state,cases,deaths,suspects,refuses,datetime,casesday,deathsday
0,35,SP,São Paulo,1418491,45758,5334,596,2020-12-24T19:06:09.179Z,0.0,0.0
1,35,SP,São Paulo,1422087,45795,5334,596,2020-12-25T21:45:45.224Z,3596.0,37.0
2,35,SP,São Paulo,1423340,45808,5334,596,2020-12-26T21:45:55.252Z,1253.0,13.0
3,35,SP,São Paulo,1426176,45863,5334,596,2020-12-27T21:46:24.258Z,2836.0,55.0
4,35,SP,São Paulo,1427752,45902,5334,596,2020-12-28T21:47:37.199Z,1576.0,39.0
...,...,...,...,...,...,...,...,...,...,...
152,12,AC,Acre,84450,1723,12,0,2021-06-16T22:37:53.773Z,46.0,2.0
153,12,AC,Acre,84487,1725,12,0,2021-06-17T22:37:58.327Z,37.0,2.0
154,12,AC,Acre,84687,1728,12,0,2021-06-18T22:37:38.352Z,200.0,3.0
155,12,AC,Acre,84812,1729,12,0,2021-06-19T21:37:01.961Z,125.0,1.0


In [7]:
#remocao de colunas que nao possuem informacoes relaventes para analise
covid_df = covid_df.drop(['suspects','refuses'], axis=1)

In [8]:
#incluindo latitude e longitude de um arquivo json
covid_df = covid_df.join(pd.read_json('./brazil_states_lat_long.json').set_index('uf'), on='uf')

In [9]:
covid_df

,uid,uf,state,cases,deaths,datetime,casesday,deathsday,latitude,longitude
0,35,SP,São Paulo,1418491,45758,2020-12-24T19:06:09.179Z,0.0,0.0,-22.19,-48.79
1,35,SP,São Paulo,1422087,45795,2020-12-25T21:45:45.224Z,3596.0,37.0,-22.19,-48.79
2,35,SP,São Paulo,1423340,45808,2020-12-26T21:45:55.252Z,1253.0,13.0,-22.19,-48.79
3,35,SP,São Paulo,1426176,45863,2020-12-27T21:46:24.258Z,2836.0,55.0,-22.19,-48.79
4,35,SP,São Paulo,1427752,45902,2020-12-28T21:47:37.199Z,1576.0,39.0,-22.19,-48.79
...,...,...,...,...,...,...,...,...,...,...
152,12,AC,Acre,84450,1723,2021-06-16T22:37:53.773Z,46.0,2.0,-8.77,-70.55
153,12,AC,Acre,84487,1725,2021-06-17T22:37:58.327Z,37.0,2.0,-8.77,-70.55
154,12,AC,Acre,84687,1728,2021-06-18T22:37:38.352Z,200.0,3.0,-8.77,-70.55
155,12,AC,Acre,84812,1729,2021-06-19T21:37:01.961Z,125.0,1.0,-8.77,-70.55


In [10]:
#Exportando arquivo para csv
covid_df.to_csv('./data_covid_project3.csv', encoding='latin-1', index = False)

In [2]:
#Link acesso Tableau Public
['https://public.tableau.com/app/profile/bruna.carolino.de.souza/viz/Livro2_16245869352840/TotalVacBr']

['https://public.tableau.com/app/profile/bruna.carolino.de.souza/viz/Livro2_16245869352840/TotalVacBr']